# Tratamento da base de dados

In [ ]:
import pandas as pd
import numpy as np
base_census = pd.read_csv('census.csv')
colunas = []
colunas.append('income')
for i in range(len(base_census.columns[:-1])):
    colunas.append(base_census.columns[i])
base_census = base_census[colunas]
base_census